In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from numpy import genfromtxt
import io
import math
import random


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
data = pd.read_excel('/content/drive/MyDrive/NNFL Assignment/data.xlsx');


In [4]:
l = np.array(data)
n = l.shape[1]
np.random.shuffle(l)
x = l[0:, 0:n-1]
#x = preprocessing.normalize(x)
#x = (x-x.mean(axis = 0))/((x.max(axis=0))-(x.min(axis=0)))
x = (x-x.mean(axis = 0))/(x.std(axis=0))
y = l[0:, n-1]

In [7]:
m = x.shape[0]
num = math.floor(m/10)
subsets = []
y_subsets = []
for i in range(10):
  subsets.append(x[i*num:(i+1)*num])
  y_subsets.append(y[i*num:(i+1)*num])
subsets.append(x[(i+1)*num:])
y_subsets.append(y[(i+1)*num:])
for j in range(10):
  print(len(subsets[j]))

341
341
341
341
341
341
341
341
341
341


ONE VS ALL

In [8]:
def one_vs_all(y):
  y_model1 = []
  y_model2 = []
  y_model3 = []
  y_model4 = []
  for ele in y:
    if (ele == 1):
      y_model1.append(0)
      y_model2.append(1)
      y_model3.append(1)
      y_model4.append(1)
    if (ele == 2):
      y_model1.append(1)
      y_model2.append(0)
      y_model3.append(1)
      y_model4.append(1)
    if (ele == 3):
      y_model1.append(1)
      y_model2.append(1)
      y_model3.append(0)
      y_model4.append(1)
    if (ele == 4):
      y_model1.append(1)
      y_model2.append(1)
      y_model3.append(1)
      y_model4.append(0)

  return y_model1,y_model2,y_model3,y_model4

In [9]:
def lor_l2_bgd(x,y):
  alpha = 0.0001
  c = 0.8286427728546842
  #c = 0.1
  iters =100
  x = np.array(x)
  n = x.shape[1]
  wt = np.ones(n)
  #J=np.zeros((iters,n-1))
  #w = np.zeros((iters,n-1))
  Z = np.dot(x,wt)
  h = sigmoid(Z)
  for i in range (0,iters):
    for j in range (0,n-1):
      x_ft = column(x,j)
      wt[j] = wt[j]*(1-(alpha*c))-alpha*sum((y-h)*x_ft)
    #w[i].append(wt[j])
    #Z = np.dot(x,wt)
    #h = sigmoid(Z)
    #J[i] = cost_fn(h,y)
  return wt

In [10]:
def l2b_weight_fn(x,y1,y2,y3,y4):
  #wt1 = lor_bgd(x,y1)
  #wt2 = lor_bgd(x,y2)
  #wt3 = lor_bgd(x,y3)

  wt1 = lor_l2_bgd(x,y1)
  wt2 = lor_l2_bgd(x,y2)
  wt3 = lor_l2_bgd(x,y3)
  wt4 = lor_l2_bgd(x,y4)

  #wt1 = lor_l1_bgd(x,y1,C)
  #wt2 = lor_l1_bgd(x,y2,C)
  #wt3 = lor_l1_bgd(x,y3,C)
  return wt1,wt2,wt3,wt4

In [11]:
def lor_bgd(x,y):
  alpha = 0.0001
  #c = 0.8286427728546842
  iters =100
  x = np.array(x)
  n = x.shape[1]
  wt = np.ones(n)
  #J=np.zeros((iters,n-1))
  Z = np.dot(x,wt)
  h = sigmoid(Z)
  for i in range (0,iters):
    for j in range (0,n-1):
      x_ft = column(x,j)
      wt[j] = wt[j]-alpha*sum((y-h)*x_ft)
  return wt

In [12]:
def b_weight_fn(x,y1,y2,y3,y4):
  wt1 = lor_bgd(x,y1)
  wt2 = lor_bgd(x,y2)
  wt3 = lor_bgd(x,y3)
  wt4 = lor_bgd(x,y4)

  #wt1 = lor_l2_bgd(x,y1,C)
  #wt2 = lor_l2_bgd(x,y2,C)
  #wt3 = lor_l2_bgd(x,y3,C)

  #wt1 = lor_l1_bgd(x,y1,C)
  #wt2 = lor_l1_bgd(x,y2,C)
  #wt3 = lor_l1_bgd(x,y3,C)
  return wt1,wt2,wt3,wt4

In [13]:
def sigmoid(z):
  return 1.0 / ( 1.0 + np.exp(-z))

In [14]:
def column(matrix, i):
    return [row[i] for row in matrix]

In [15]:
def b_weight_fn(x,y1,y2,y3,y4):
  wt1 = lor_bgd(x,y1)
  wt2 = lor_bgd(x,y2)
  wt3 = lor_bgd(x,y3)
  wt4 = lor_bgd(x,y4)

  #wt1 = lor_l2_bgd(x,y1,C)
  #wt2 = lor_l2_bgd(x,y2,C)
  #wt3 = lor_l2_bgd(x,y3,C)

  #wt1 = lor_l1_bgd(x,y1,C)
  #wt2 = lor_l1_bgd(x,y2,C)
  #wt3 = lor_l1_bgd(x,y3,C)
  return wt1,wt2,wt3,wt4

In [16]:
ovr_acc = 0
acc1 = 0
acc2 = 0
acc3 = 0
acc4 = 0
for f in range(10):
  X_train = []
  Y_train = []
  #a_outputs = []
  pred_op = []
  p_outputs = []
  arr1 = np.array(subsets[f])
  X_test = arr1.tolist()
  Y_test = y_subsets[f]
  for k in range(10):
    if k!=f:
      arr = np.array(subsets[k])
      list1 = arr.tolist()
      X_train.extend(list1)
      Y_train.extend(y_subsets[k])
  y_tr1,y_tr2,y_tr3,y_tr4 = one_vs_all(Y_train)

  w1,w2,w3,w4 = b_weight_fn(X_train,y_tr1,y_tr2,y_tr3,y_tr4)
  for b in range(len(X_test)):
    pred_op = []

    Z1 = np.dot(X_test[b],w1)
    pred_op.append(sigmoid(Z1))
    Z2 = np.dot(X_test[b],w2)
    pred_op.append(sigmoid(Z2))
    Z3 = np.dot(X_test[b],w3)
    pred_op.append(sigmoid(Z3))
    Z4 = np.dot(X_test[b],w4)
    pred_op.append(sigmoid(Z4))
    
    
    pred_op = np.array(pred_op)
    p_outputs.append(np.argmax(pred_op)+1)
    pred_op = pred_op.tolist()  

  print( 'Fold ' + str(f+1))
  y_actual = pd.Series(Y_test, name= 'Actual' )
  y_pred = pd.Series(p_outputs, name= 'Predicted' )
  confmat = pd.crosstab(y_actual,y_pred)
  print(confmat)
  confmat = np.asarray(confmat)
  class_acc = np.zeros(4)
  for i in range(4):
    num = confmat[i][i]
    s =0
    for j in range(4):
      s += confmat[i][j]
    class_acc[i]= num/s
  #print(class_acc)
  acc = np.trace(confmat)/np.sum(confmat)
  for c in range(4):
    print( 'Accuracy of class'+ str(c+1)+' : ' + str(class_acc[c]*100))
  print( 'Overall Accuracy : ' + str(acc*100))
  ovr_acc += acc
  acc1 += class_acc[0]
  acc2 += class_acc[1]
  acc3 += class_acc[2]
  acc4 += class_acc[3]
  print(p_outputs)
  print(Y_test)
print( 'Average Accuracy : ' + str(ovr_acc*20))
print( 'Average Accuracy of class 1 : ' + str(acc1*20))
print( 'Average Accuracy of class 2 : ' + str(acc2*20))
print( 'Average Accuracy of class 3 : ' + str(acc3*20))
print( 'Average Accuracy of class 4 : ' + str(acc4*20))
  #print(p_outputs)
  #print(Y_test)

Fold 1
Predicted   1   2   3   4
Actual                   
1.0        54  11   9  11
2.0        64   7  11  14
3.0        50   4   3  18
4.0        47  14   6  18
Accuracy of class1 : 63.52941176470588
Accuracy of class2 : 7.291666666666667
Accuracy of class3 : 4.0
Accuracy of class4 : 21.176470588235293
Overall Accuracy : 24.04692082111437
[4, 1, 1, 4, 3, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 4, 4, 1, 1, 1, 3, 4, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 2, 1, 1, 4, 1, 4, 4, 1, 4, 1, 1, 2, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 4, 1, 1, 1, 1, 2, 4, 1, 1, 3, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 4, 1, 1, 4, 1, 3, 1, 1, 1, 2, 2, 2, 3, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 4, 1, 1, 3, 1, 4, 3, 1, 1, 1, 4, 1, 1, 3, 1, 2, 1, 3, 1, 1, 2, 4, 1, 1, 1, 1, 2, 1, 1, 4, 1, 1, 2, 1, 2, 1, 2, 4, 4, 4, 4, 1, 1, 1, 1, 1, 2, 1, 1, 4, 1, 4, 1, 4, 1, 1, 2, 2, 1, 3, 1, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,

In [17]:
def l2b_weight_fn(x,y1,y2,y3,y4):
  #wt1 = lor_bgd(x,y1)
  #wt2 = lor_bgd(x,y2)
  #wt3 = lor_bgd(x,y3)

  wt1 = lor_l2_bgd(x,y1)
  wt2 = lor_l2_bgd(x,y2)
  wt3 = lor_l2_bgd(x,y3)
  wt4 = lor_l2_bgd(x,y4)

  #wt1 = lor_l1_bgd(x,y1,C)
  #wt2 = lor_l1_bgd(x,y2,C)
  #wt3 = lor_l1_bgd(x,y3,C)
  return wt1,wt2,wt3,wt4

In [18]:
ovr_acc = 0
acc1 = 0
acc2 = 0
acc3 = 0
acc3 = 0
for f in range(10):
  X_train = []
  Y_train = []
  #a_outputs = []
  pred_op = []
  p_outputs = []
  arr1 = np.array(subsets[f])
  X_test = arr1.tolist()
  Y_test = y_subsets[f]
  for k in range(10):
    if k!=f:
      arr = np.array(subsets[k])
      list1 = arr.tolist()
      X_train.extend(list1)
      Y_train.extend(y_subsets[k])
  y_tr1,y_tr2,y_tr3,y_tr4 = one_vs_all(Y_train)

  w1,w2,w3,w4 = l2b_weight_fn(X_train,y_tr1,y_tr2,y_tr3,y_tr4)
  for b in range(len(X_test)):
    pred_op = []

    Z1 = np.dot(X_test[b],w1)
    pred_op.append(sigmoid(Z1))
    Z2 = np.dot(X_test[b],w2)
    pred_op.append(sigmoid(Z2))
    Z3 = np.dot(X_test[b],w3)
    pred_op.append(sigmoid(Z3))
    Z4 = np.dot(X_test[b],w4)
    pred_op.append(sigmoid(Z4))


    pred_op = np.array(pred_op)
    p_outputs.append(np.argmax(pred_op)+1)
    pred_op = pred_op.tolist()   

  print( 'Fold ' + str(f+1))
  y_actual = pd.Series(Y_test, name= 'Actual' )
  y_pred = pd.Series(p_outputs, name= 'Predicted' )
  confmat = pd.crosstab(y_actual,y_pred)
  print(confmat)
  confmat = np.asarray(confmat)
  class_acc = np.zeros(4)
  for i in range(4):
    num = confmat[i][i]
    s =0
    for j in range(4):
      s += confmat[i][j]
    class_acc[i]= num/s
  #print(class_acc)
  acc = np.trace(confmat)/np.sum(confmat)
  for c in range(4):
    print( 'Accuracy of class'+ str(c+1)+' : ' + str(class_acc[c]*100))
  print( 'Overall Accuracy : ' + str(acc*100))
  ovr_acc += acc
  acc1 += class_acc[0]
  acc2 += class_acc[1]
  acc3 += class_acc[2]
  acc4 += class_acc[3]
  print(p_outputs)
  print(Y_test)
print( 'Average Accuracy : ' + str(ovr_acc*20))
print( 'Average Accuracy of class 1 : ' + str(acc1*20))
print( 'Average Accuracy of class 2 : ' + str(acc2*20))
print( 'Average Accuracy of class 3 : ' + str(acc3*20))
print( 'Average Accuracy of class 4 : ' + str(acc4*20))
  #print(p_outputs)
  #print(Y_test)

Fold 1
Predicted   1   2   3   4
Actual                   
1.0        54  11   9  11
2.0        63   8  11  14
3.0        50   4   3  18
4.0        47  14   6  18
Accuracy of class1 : 63.52941176470588
Accuracy of class2 : 8.333333333333332
Accuracy of class3 : 4.0
Accuracy of class4 : 21.176470588235293
Overall Accuracy : 24.34017595307918
[4, 1, 1, 4, 3, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 4, 4, 1, 1, 1, 3, 4, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 2, 1, 1, 4, 1, 4, 4, 1, 4, 1, 1, 2, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 4, 1, 1, 1, 1, 2, 4, 1, 1, 3, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 4, 1, 1, 4, 1, 3, 1, 1, 1, 2, 2, 2, 3, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 4, 1, 1, 3, 1, 4, 3, 1, 1, 1, 4, 1, 1, 3, 1, 2, 1, 3, 1, 1, 2, 4, 1, 1, 1, 1, 2, 1, 1, 4, 1, 1, 2, 1, 2, 1, 2, 4, 4, 4, 4, 1, 1, 1, 1, 1, 2, 1, 1, 4, 1, 4, 1, 4, 1, 1, 2, 2, 1, 3, 1, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1,

In [19]:
def lor_l1_bgd(x,y):
  alpha = 0.0001
  c = 0.83
  iters =100
  x = np.array(x)
  n = x.shape[1]
  wt = np.ones(n)
  #J=np.zeros((iters,n-1))
  Z = np.dot(x,wt)
  h = sigmoid(Z)
  for i in range (0,iters):
    for j in range (0,n-1):
      x_ft = column(x,j)
      wt[j] = wt[j]-alpha*sum((y-h)*x_ft)-(alpha*c*0.5)*np.sign(wt[j])
  return wt

In [20]:
def l1b_weight_fn(x,y1,y2,y3,y4):
  #wt1 = lor_bgd(x,y1)
  #wt2 = lor_bgd(x,y2)
  #wt3 = lor_bgd(x,y3)

  #wt1 = lor_l2_bgd(x,y1,C)
  #wt2 = lor_l2_bgd(x,y2,C)
  #wt3 = lor_l2_bgd(x,y3,C)

  wt1 = lor_l1_bgd(x,y1)
  wt2 = lor_l1_bgd(x,y2)
  wt3 = lor_l1_bgd(x,y3)
  wt4 = lor_l1_bgd(x,y4)

  return wt1,wt2,wt3,wt4

In [21]:
ovr_acc = 0
acc1 = 0
acc2 = 0
acc3 = 0
acc4 = 0
for f in range(10):
  X_train = []
  Y_train = []
  #a_outputs = []
  pred_op = []
  p_outputs = []
  arr1 = np.array(subsets[f])
  X_test = arr1.tolist()
  Y_test = y_subsets[f]
  for k in range(10):
    if k!=f:
      arr = np.array(subsets[k])
      list1 = arr.tolist()
      X_train.extend(list1)
      Y_train.extend(y_subsets[k])
  y_tr1,y_tr2,y_tr3,y_tr4 = one_vs_all(Y_train)

  w1,w2,w3,w4 = l1b_weight_fn(X_train,y_tr1,y_tr2,y_tr3,y_tr4)
  for b in range(len(X_test)):
    pred_op = []

    Z1 = np.dot(X_test[b],w1)
    pred_op.append(sigmoid(Z1))
    Z2 = np.dot(X_test[b],w2)
    pred_op.append(sigmoid(Z2))
    Z3 = np.dot(X_test[b],w3)
    pred_op.append(sigmoid(Z3))
    Z4 = np.dot(X_test[b],w4)
    pred_op.append(sigmoid(Z4))

    pred_op = np.array(pred_op)
    p_outputs.append(np.argmax(pred_op)+1)
    pred_op = pred_op.tolist()    
  print( 'Fold ' + str(f+1))
  y_actual = pd.Series(Y_test, name= 'Actual' )
  y_pred = pd.Series(p_outputs, name= 'Predicted' )
  confmat = pd.crosstab(y_actual,y_pred)
  print(confmat)
  confmat = np.asarray(confmat)
  class_acc = np.zeros(4)
  for i in range(4):
    num = confmat[i][i]
    s =0
    for j in range(4):
      s += confmat[i][j]
    class_acc[i]= num/s
  #print(class_acc)
  acc = np.trace(confmat)/np.sum(confmat)
  for c in range(4):
    print( 'Accuracy of class'+ str(c+1)+' : ' + str(class_acc[c]*100))
  print( 'Overall Accuracy : ' + str(acc*100))
  ovr_acc += acc
  acc1 += class_acc[0]
  acc2 += class_acc[1]
  acc3 += class_acc[2]
  acc4 += class_acc[3]
  print(p_outputs)
  print(Y_test)
print( 'Average Accuracy : ' + str(ovr_acc*20))
print( 'Average Accuracy of class 1 : ' + str(acc1*20))
print( 'Average Accuracy of class 2 : ' + str(acc2*20))
print( 'Average Accuracy of class 3 : ' + str(acc3*20))
print( 'Average Accuracy of class 4 : ' + str(acc4*20))
  #print(p_outputs)
  #print(Y_test)

Fold 1
Predicted   1   2   3   4
Actual                   
1.0        54  11   9  11
2.0        64   7  11  14
3.0        50   4   3  18
4.0        47  14   6  18
Accuracy of class1 : 63.52941176470588
Accuracy of class2 : 7.291666666666667
Accuracy of class3 : 4.0
Accuracy of class4 : 21.176470588235293
Overall Accuracy : 24.04692082111437
[4, 1, 1, 4, 3, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 4, 4, 1, 1, 1, 3, 4, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 2, 1, 1, 4, 1, 4, 4, 1, 4, 1, 1, 2, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 4, 1, 1, 1, 1, 2, 4, 1, 1, 3, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 4, 1, 1, 4, 1, 3, 1, 1, 1, 2, 2, 2, 3, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 4, 1, 1, 3, 1, 4, 3, 1, 1, 1, 4, 1, 1, 3, 1, 2, 1, 3, 1, 1, 2, 4, 1, 1, 1, 1, 2, 1, 1, 4, 1, 1, 2, 1, 2, 1, 2, 4, 4, 4, 4, 1, 1, 1, 1, 1, 2, 1, 1, 4, 1, 4, 1, 4, 1, 1, 2, 2, 1, 3, 1, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,

In [23]:
def lor_l2_sgd(x,y):
  alpha = 0.0001
  c = 0.83
  iters =100
  x = np.array(x)
  n = x.shape[1]
  wt = np.ones(n)
  #J=np.zeros((iters,n-1))
  #w = np.zeros((iters,n-1))
  #Z = np.dot(x,wt)
  #h = sigmoid(Z)
  for i in range (0,iters):
    randindex = np.random.randint(len(y))
    ind_x = x[randindex:randindex+1]
    ind_y = y[randindex:randindex+1]
    for j in range (0,n-1):
      x_ft = column(ind_x,j)
      wt[j] = wt[j]*(1-(alpha*c))-alpha*(ind_y-sigmoid(np.dot(ind_x,wt)))*x_ft
  return wt

In [24]:
def l2s_weight_fn(x,y1,y2,y3,y4):
  #wt1 = lor_bgd(x,y1)
  #wt2 = lor_bgd(x,y2)
  #wt3 = lor_bgd(x,y3)

  wt1 = lor_l2_sgd(x,y1)
  wt2 = lor_l2_sgd(x,y2)
  wt3 = lor_l2_sgd(x,y3)
  wt4 = lor_l2_sgd(x,y4)

  #wt1 = lor_l1_bgd(x,y1,C)
  #wt2 = lor_l1_bgd(x,y2,C)
  #wt3 = lor_l1_bgd(x,y3,C)
  return wt1,wt2,wt3,wt4

In [25]:
ovr_acc = 0
acc1 = 0
acc2 = 0
acc3 = 0
acc4 = 0
for f in range(10):
  X_train = []
  Y_train = []
  #a_outputs = []
  pred_op = []
  p_outputs = []
  arr1 = np.array(subsets[f])
  X_test = arr1.tolist()
  Y_test = y_subsets[f]
  for k in range(10):
    if k!=f:
      arr = np.array(subsets[k])
      list1 = arr.tolist()
      X_train.extend(list1)
      Y_train.extend(y_subsets[k])
  y_tr1,y_tr2,y_tr3,y_tr4 = one_vs_all(Y_train)

  w1,w2,w3,w4 = l2s_weight_fn(X_train,y_tr1,y_tr2,y_tr3,y_tr4)
  for b in range(len(X_test)):
    pred_op = []

    Z1 = np.dot(X_test[b],w1)
    pred_op.append(sigmoid(Z1))
    Z2 = np.dot(X_test[b],w2)
    pred_op.append(sigmoid(Z2))
    Z3 = np.dot(X_test[b],w3)
    pred_op.append(sigmoid(Z3))
    Z4 = np.dot(X_test[b],w4)
    pred_op.append(sigmoid(Z4))
    
    pred_op = np.array(pred_op)
    p_outputs.append(np.argmax(pred_op)+1)
    pred_op = pred_op.tolist()    
  print( 'Fold ' + str(f+1))
  y_actual = pd.Series(Y_test, name= 'Actual' )
  y_pred = pd.Series(p_outputs, name= 'Predicted' )
  confmat = pd.crosstab(y_actual,y_pred)
  print(confmat)
  confmat = np.asarray(confmat)
  class_acc = np.zeros(4)
  for i in range(4):
    num = confmat[i][i]
    s =0
    for j in range(4):
      s += confmat[i][j]
    class_acc[i]= num/s
  #print(class_acc)
  acc = np.trace(confmat)/np.sum(confmat)
  for c in range(4):
    print( 'Accuracy of class'+ str(c+1)+' : ' + str(class_acc[c]*100))
  print( 'Overall Accuracy : ' + str(acc*100))
  ovr_acc += acc
  acc1 += class_acc[0]
  acc2 += class_acc[1]
  acc3 += class_acc[2]
  acc4 += class_acc[3]
  print(p_outputs)
  print(Y_test)
print( 'Average Accuracy : ' + str(ovr_acc*20))
print( 'Average Accuracy of class 1 : ' + str(acc1*20))
print( 'Average Accuracy of class 2 : ' + str(acc2*20))
print( 'Average Accuracy of class 3 : ' + str(acc3*20))
print( 'Average Accuracy of class 4 : ' + str(acc4*20))
  #print(p_outputs)
  #print(Y_test)


Fold 1
Predicted   1   2   3   4
Actual                   
1.0        27  14  19  25
2.0        32  15  20  29
3.0        23  18  18  16
4.0        23  23  21  18
Accuracy of class1 : 31.76470588235294
Accuracy of class2 : 15.625
Accuracy of class3 : 24.0
Accuracy of class4 : 21.176470588235293
Overall Accuracy : 22.87390029325513
[3, 2, 2, 4, 3, 2, 1, 3, 4, 1, 4, 1, 2, 4, 3, 4, 1, 1, 2, 4, 1, 4, 1, 3, 3, 1, 2, 4, 1, 1, 4, 2, 4, 4, 1, 2, 1, 2, 1, 3, 1, 3, 3, 2, 3, 4, 3, 2, 1, 4, 1, 1, 3, 1, 1, 4, 1, 1, 3, 2, 3, 1, 4, 2, 1, 2, 4, 1, 1, 1, 3, 1, 1, 2, 2, 4, 3, 4, 2, 2, 1, 3, 3, 3, 1, 4, 3, 1, 4, 2, 3, 1, 1, 4, 3, 4, 1, 2, 1, 4, 2, 2, 1, 1, 1, 4, 1, 3, 4, 3, 1, 1, 2, 4, 2, 4, 3, 4, 3, 4, 2, 3, 1, 1, 3, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 4, 1, 4, 4, 1, 4, 4, 3, 1, 4, 3, 1, 2, 4, 2, 3, 1, 3, 4, 3, 2, 4, 4, 2, 1, 4, 2, 2, 4, 2, 2, 1, 1, 2, 4, 1, 3, 1, 2, 2, 1, 1, 4, 1, 4, 2, 2, 4, 1, 4, 4, 1, 4, 3, 4, 3, 3, 1, 4, 3, 1, 3, 2, 1, 4, 2, 1, 4, 1, 2, 2, 3, 3, 3, 1, 4, 3, 1, 2, 3, 

In [41]:
def lor_l1_sgd(x,y):
  alpha = 0.0001
  c = 0.8286427728546842
  iters =100
  x = np.array(x)
  n = x.shape[1]
  wt = np.ones(n)
  #J=np.zeros((iters,n-1))
  #Z = np.dot(x,wt)
  #h = sigmoid(Z)
  for i in range (0,iters):
    randindex = np.random.randint(len(y))
    ind_x = x[randindex:randindex+1]
    ind_y = y[randindex:randindex+1]
    for j in range (0,n-1):
      x_ft = column(ind_x,j)
      wt[j] = wt[j]-(alpha*(ind_y-sigmoid(np.dot(ind_x,wt)))*x_ft)-(alpha*c*0.5)*np.sign(wt[j])
  return wt

In [43]:
def l1s_weight_fn(x,y1,y2,y3,y4):
  #wt1 = lor_bgd(x,y1)
  #wt2 = lor_bgd(x,y2)
  #wt3 = lor_bgd(x,y3)

  #wt1 = lor_l2_bgd(x,y1,C)
  #wt2 = lor_l2_bgd(x,y2,C)
  #wt3 = lor_l2_bgd(x,y3,C)

  wt1 = lor_l1_sgd(x,y1)
  wt2 = lor_l1_sgd(x,y2)
  wt3 = lor_l1_sgd(x,y3)
  wt4 = lor_l1_sgd(x,y4)

  return wt1,wt2,wt3,wt4

In [44]:
ovr_acc = 0
acc1 = 0
acc2 = 0
acc3 = 0
acc4 = 0
for f in range(10):
  X_train = []
  Y_train = []
  #a_outputs = []
  pred_op = []
  p_outputs = []
  arr1 = np.array(subsets[f])
  X_test = arr1.tolist()
  Y_test = y_subsets[f]
  for k in range(10):
    if k!=f:
      arr = np.array(subsets[k])
      list1 = arr.tolist()
      X_train.extend(list1)
      Y_train.extend(y_subsets[k])
  y_tr1,y_tr2,y_tr3,y_tr4 = one_vs_all(Y_train)

  w1,w2,w3,w4 = l1s_weight_fn(X_train,y_tr1,y_tr2,y_tr3,y_tr4)
  for b in range(len(X_test)):
    pred_op = []

    Z1 = np.dot(X_test[b],w1)
    pred_op.append(sigmoid(Z1))
    Z2 = np.dot(X_test[b],w2)
    pred_op.append(sigmoid(Z2))
    Z3 = np.dot(X_test[b],w3)
    pred_op.append(sigmoid(Z3))
    Z4 = np.dot(X_test[b],w4)
    pred_op.append(sigmoid(Z4))
    
    pred_op = np.array(pred_op)
    p_outputs.append(np.argmax(pred_op)+1)
    pred_op = pred_op.tolist()    
  print( 'Fold ' + str(f+1))
  y_actual = pd.Series(Y_test, name= 'Actual' )
  y_pred = pd.Series(p_outputs, name= 'Predicted' )
  confmat = pd.crosstab(y_actual,y_pred)
  print(confmat)
  confmat = np.asarray(confmat)
  class_acc = np.zeros(4)
  for i in range(4):
    num = confmat[i][i]
    s =0
    for j in range(4):
      s += confmat[i][j]
    class_acc[i]= num/s
  #print(class_acc)
  acc = np.trace(confmat)/np.sum(confmat)
  for c in range(4):
    print( 'Accuracy of class'+ str(c+1)+' : ' + str(class_acc[c]*100))
  print( 'Overall Accuracy : ' + str(acc*100))
  ovr_acc += acc
  acc1 += class_acc[0]
  acc2 += class_acc[1]
  acc3 += class_acc[2]
  acc4 += class_acc[3]
  print(p_outputs)
  print(Y_test)
print( 'Average Accuracy : ' + str(ovr_acc*20))
print( 'Average Accuracy of class 1 : ' + str(acc1*20))
print( 'Average Accuracy of class 2 : ' + str(acc2*20))
print( 'Average Accuracy of class 3 : ' + str(acc3*20))
print( 'Average Accuracy of class 4 : ' + str(acc4*20))
  #print(p_outputs)
  #print(Y_test)


Fold 1
Predicted   1   2   3   4
Actual                   
1.0        29  18  23  15
2.0        27  18  33  18
3.0        16  17  19  23
4.0        26  17  28  14
Accuracy of class1 : 34.11764705882353
Accuracy of class2 : 18.75
Accuracy of class3 : 25.333333333333336
Accuracy of class4 : 16.470588235294116
Overall Accuracy : 23.46041055718475
[4, 4, 1, 1, 3, 1, 4, 3, 3, 3, 3, 3, 3, 4, 2, 1, 4, 4, 3, 3, 2, 1, 2, 3, 1, 3, 4, 3, 1, 3, 2, 3, 2, 1, 3, 4, 3, 2, 2, 1, 1, 1, 3, 1, 4, 1, 3, 2, 3, 2, 4, 3, 3, 3, 2, 2, 3, 4, 2, 2, 3, 4, 1, 1, 3, 4, 3, 1, 1, 3, 1, 4, 4, 1, 1, 3, 2, 2, 1, 3, 3, 4, 4, 4, 3, 1, 2, 3, 3, 1, 1, 4, 4, 3, 1, 2, 3, 2, 2, 3, 1, 4, 3, 1, 3, 2, 3, 1, 1, 2, 1, 4, 1, 2, 1, 3, 4, 3, 4, 3, 1, 1, 4, 4, 3, 4, 3, 2, 4, 3, 2, 2, 3, 4, 4, 1, 3, 3, 2, 1, 4, 3, 2, 3, 1, 3, 4, 1, 2, 1, 3, 2, 2, 4, 3, 3, 1, 4, 2, 4, 2, 4, 3, 2, 3, 3, 4, 2, 1, 4, 2, 4, 3, 3, 1, 1, 3, 3, 3, 4, 4, 1, 3, 4, 3, 4, 4, 1, 3, 3, 4, 1, 2, 3, 2, 1, 4, 4, 1, 1, 4, 1, 2, 3, 1, 2, 1, 3, 3, 3, 2, 1, 4, 1, 4, 2, 2, 1,

In [29]:
def lor_sgd(x,y):
  alpha = 0.0001
  #c = 0.8286427728546842
  iters =100
  x = np.array(x)
  n = x.shape[1]
  wt = np.ones(n)
  #J=np.zeros((iters,n-1))
  #Z = np.dot(x,wt)
  #h = sigmoid(Z)
  for i in range (0,iters):
    randindex = np.random.randint(len(y))
    ind_x = x[randindex:randindex+1]
    ind_y = y[randindex:randindex+1]
    for j in range (0,n-1):
      x_ft = column(ind_x,j)
      wt[j] = wt[j]-alpha*(ind_y-sigmoid(np.dot(ind_x,wt)))*x_ft
  return wt

In [30]:
def s_weight_fn(x,y1,y2,y3,y4):
  wt1 = lor_sgd(x,y1)
  wt2 = lor_sgd(x,y2)
  wt3 = lor_sgd(x,y3)
  wt4 = lor_sgd(x,y4)

  #wt1 = lor_l2_bgd(x,y1,C)
  #wt2 = lor_l2_bgd(x,y2,C)
  #wt3 = lor_l2_bgd(x,y3,C)

  #wt1 = lor_l1_bgd(x,y1,C)
  #wt2 = lor_l1_bgd(x,y2,C)
  #wt3 = lor_l1_bgd(x,y3,C)
  return wt1,wt2,wt3,wt4

In [31]:
ovr_acc = 0
acc1 = 0
acc2 = 0
acc3 = 0
acc4 = 0
for f in range(10):
  X_train = []
  Y_train = []
  #a_outputs = []
  pred_op = []
  p_outputs = []
  arr1 = np.array(subsets[f])
  X_test = arr1.tolist()
  Y_test = y_subsets[f]
  for k in range(10):
    if k!=f:
      arr = np.array(subsets[k])
      list1 = arr.tolist()
      X_train.extend(list1)
      Y_train.extend(y_subsets[k])
  y_tr1,y_tr2,y_tr3,y_tr4 = one_vs_all(Y_train)

  w1,w2,w3,w4 = s_weight_fn(X_train,y_tr1,y_tr2,y_tr3,y_tr4)
  for b in range(len(X_test)):
    pred_op = []

    Z1 = np.dot(X_test[b],w1)
    pred_op.append(sigmoid(Z1))
    Z2 = np.dot(X_test[b],w2)
    pred_op.append(sigmoid(Z2))
    Z3 = np.dot(X_test[b],w3)
    pred_op.append(sigmoid(Z3))
    Z4 = np.dot(X_test[b],w4)
    pred_op.append(sigmoid(Z4))
    
    pred_op = np.array(pred_op)
    p_outputs.append(np.argmax(pred_op)+1)
    pred_op = pred_op.tolist()    
  print( 'Fold ' + str(f+1))
  y_actual = pd.Series(Y_test, name= 'Actual' )
  y_pred = pd.Series(p_outputs, name= 'Predicted' )
  confmat = pd.crosstab(y_actual,y_pred)
  print(confmat)
  confmat = np.asarray(confmat)
  class_acc = np.zeros(4)
  for i in range(4):
    num = confmat[i][i]
    s =0
    for j in range(4):
      s += confmat[i][j]
    class_acc[i]= num/s
  #print(class_acc)
  acc = np.trace(confmat)/np.sum(confmat)
  for c in range(4):
    print( 'Accuracy of class'+ str(c+1)+' : ' + str(class_acc[c]*100))
  print( 'Overall Accuracy : ' + str(acc*100))
  ovr_acc += acc
  acc1 += class_acc[0]
  acc2 += class_acc[1]
  acc3 += class_acc[2]
  acc4 += class_acc[3]
  print(p_outputs)
  print(Y_test)
print( 'Average Accuracy : ' + str(ovr_acc*20))
print( 'Average Accuracy of class 1 : ' + str(acc1*20))
print( 'Average Accuracy of class 2 : ' + str(acc2*20))
print( 'Average Accuracy of class 3 : ' + str(acc3*20))
print( 'Average Accuracy of class 4 : ' + str(acc4*20))
  #print(p_outputs)
  #print(Y_test)


Fold 1
Predicted   1   2   3   4
Actual                   
1.0        30  23  14  18
2.0        26  32  18  20
3.0        17  23  18  17
4.0        19  28  18  20
Accuracy of class1 : 35.294117647058826
Accuracy of class2 : 33.33333333333333
Accuracy of class3 : 24.0
Accuracy of class4 : 23.52941176470588
Overall Accuracy : 29.32551319648094
[1, 3, 2, 1, 4, 3, 3, 4, 2, 1, 4, 3, 2, 3, 3, 1, 2, 3, 2, 1, 2, 2, 3, 1, 1, 2, 1, 2, 2, 1, 4, 2, 3, 2, 4, 4, 4, 3, 1, 2, 1, 2, 1, 3, 4, 2, 2, 3, 4, 1, 4, 2, 1, 4, 2, 4, 2, 1, 4, 3, 1, 4, 4, 3, 1, 3, 4, 1, 1, 2, 4, 4, 1, 4, 2, 1, 3, 3, 1, 2, 1, 2, 1, 1, 4, 2, 4, 2, 2, 2, 1, 1, 1, 4, 1, 4, 2, 1, 3, 2, 2, 4, 1, 2, 2, 4, 2, 2, 4, 4, 2, 4, 4, 3, 2, 2, 3, 2, 3, 4, 2, 1, 1, 4, 4, 3, 1, 4, 3, 4, 3, 3, 4, 3, 3, 3, 2, 1, 1, 2, 3, 2, 4, 3, 2, 2, 4, 2, 1, 1, 2, 1, 4, 3, 2, 2, 2, 1, 1, 4, 4, 3, 2, 3, 2, 2, 2, 1, 2, 3, 2, 3, 2, 4, 1, 3, 2, 4, 2, 2, 3, 2, 2, 1, 1, 1, 3, 2, 2, 2, 4, 2, 4, 4, 3, 2, 3, 3, 1, 1, 4, 4, 4, 4, 1, 3, 2, 2, 4, 2, 3, 3, 3, 2, 3, 1, 1, 2, 2

In [32]:
def lor_l2_mbgd(x,y):
  alpha = 0.0001
  #c = 0.83
  c = 0.8286427728546842
  iters =100
  x = np.array(x)
  n = x.shape[1]
  wt = np.ones(n)
  #J=np.zeros((iters,n-1))
  #w = np.zeros((iters,n-1))
  batch_size = 20
  for i in range (0,iters):
    randindex = np.random.randint(len(y))
    ind_x = x[randindex:randindex+batch_size]
    ind_y = y[randindex:randindex+batch_size]
    for j in range (0,n-1):
      x_ft = column(ind_x,j)
      Z =  np.dot(ind_x,wt)
      h = sigmoid(Z)
      wt[j] = wt[j]*(1-(alpha*c))-alpha*sum((ind_y-h)*x_ft)
  return wt

In [33]:
def l2m_weight_fn(x,y1,y2,y3,y4):
  #wt1 = lor_bgd(x,y1)
  #wt2 = lor_bgd(x,y2)
  #wt3 = lor_bgd(x,y3)

  wt1 = lor_l2_mbgd(x,y1)
  wt2 = lor_l2_mbgd(x,y2)
  wt3 = lor_l2_mbgd(x,y3)
  wt4 = lor_l2_mbgd(x,y4)

  #wt1 = lor_l1_bgd(x,y1,C)
  #wt2 = lor_l1_bgd(x,y2,C)
  #wt3 = lor_l1_bgd(x,y3,C)
  return wt1,wt2,wt3,wt4

In [34]:
ovr_acc = 0
acc1 = 0
acc2 = 0
acc3 = 0
acc4 = 0
for f in range(10):
  X_train = []
  Y_train = []
  #a_outputs = []
  pred_op = []
  p_outputs = []
  arr1 = np.array(subsets[f])
  X_test = arr1.tolist()
  Y_test = y_subsets[f]
  for k in range(10):
    if k!=f:
      arr = np.array(subsets[k])
      list1 = arr.tolist()
      X_train.extend(list1)
      Y_train.extend(y_subsets[k])
  y_tr1,y_tr2,y_tr3,y_tr4 = one_vs_all(Y_train)

  w1,w2,w3,w4 = l2m_weight_fn(X_train,y_tr1,y_tr2,y_tr3,y_tr4)
  for b in range(len(X_test)):
    pred_op = []

    Z1 = np.dot(X_test[b],w1)
    pred_op.append(sigmoid(Z1))
    Z2 = np.dot(X_test[b],w2)
    pred_op.append(sigmoid(Z2))
    Z3 = np.dot(X_test[b],w3)
    pred_op.append(sigmoid(Z3))
    Z4 = np.dot(X_test[b],w4)
    pred_op.append(sigmoid(Z4))
    
    pred_op = np.array(pred_op)
    p_outputs.append(np.argmax(pred_op)+1)
    pred_op = pred_op.tolist()    
  print( 'Fold ' + str(f+1))
  y_actual = pd.Series(Y_test, name= 'Actual' )
  y_pred = pd.Series(p_outputs, name= 'Predicted' )
  confmat = pd.crosstab(y_actual,y_pred)
  print(confmat)
  confmat = np.asarray(confmat)
  class_acc = np.zeros(4)
  for i in range(4):
    num = confmat[i][i]
    s =0
    for j in range(4):
      s += confmat[i][j]
    class_acc[i]= num/s
  #print(class_acc)
  acc = np.trace(confmat)/np.sum(confmat)
  for c in range(4):
    print( 'Accuracy of class'+ str(c+1)+' : ' + str(class_acc[c]*100))
  print( 'Overall Accuracy : ' + str(acc*100))
  ovr_acc += acc
  acc1 += class_acc[0]
  acc2 += class_acc[1]
  acc3 += class_acc[2]
  acc4 += class_acc[3]
  print(p_outputs)
  print(Y_test)
print( 'Average Accuracy : ' + str(ovr_acc*20))
print( 'Average Accuracy of class 1 : ' + str(acc1*20))
print( 'Average Accuracy of class 2 : ' + str(acc2*20))
print( 'Average Accuracy of class 3 : ' + str(acc3*20))
print( 'Average Accuracy of class 4 : ' + str(acc4*20))
  #print(p_outputs)
  #print(Y_test)


Fold 1
Predicted   1   2   3   4
Actual                   
1.0        30  10  15  30
2.0        32  16  10  38
3.0        27  13  12  23
4.0        20  13  15  37
Accuracy of class1 : 35.294117647058826
Accuracy of class2 : 16.666666666666664
Accuracy of class3 : 16.0
Accuracy of class4 : 43.529411764705884
Overall Accuracy : 27.859237536656888
[4, 1, 1, 4, 4, 1, 1, 4, 3, 1, 4, 3, 2, 3, 1, 1, 4, 1, 4, 4, 2, 2, 1, 4, 4, 2, 1, 3, 2, 1, 1, 4, 1, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 1, 4, 2, 4, 1, 1, 1, 2, 1, 3, 1, 4, 1, 3, 4, 4, 1, 4, 1, 4, 3, 4, 1, 1, 4, 4, 3, 4, 3, 1, 2, 1, 2, 1, 1, 4, 2, 3, 2, 4, 2, 4, 1, 4, 4, 1, 1, 3, 1, 1, 4, 4, 2, 1, 2, 4, 3, 2, 4, 3, 2, 2, 3, 4, 1, 2, 2, 1, 4, 1, 4, 1, 1, 4, 1, 3, 2, 1, 1, 1, 4, 1, 1, 1, 1, 1, 4, 3, 1, 4, 4, 1, 4, 3, 1, 3, 4, 4, 4, 1, 4, 4, 3, 3, 2, 2, 4, 3, 4, 1, 1, 3, 1, 2, 2, 4, 4, 1, 1, 4, 1, 2, 1, 3, 4, 4, 4, 4, 4, 3, 1, 1, 4, 2, 1, 4, 4, 1, 4, 3, 4, 1, 2, 3, 3, 1, 2, 3, 2, 4, 4, 2, 1, 1, 4, 1, 2, 2, 3, 4, 2, 4, 2, 1, 4, 1, 3, 1, 4

In [35]:
def lor_l1_mbgd(x,y):
  alpha = 0.0001
  c = 0.8286427728546842
  iters =100
  x = np.array(x)
  n = x.shape[1]
  wt = np.ones(n)
  #J=np.zeros((iters,n-1))
  batch_size = 20
  for i in range (0,iters):
    randindex = np.random.randint(len(y))
    ind_x = x[randindex:randindex+batch_size]
    ind_y = y[randindex:randindex+batch_size]
    for j in range (0,n-1):
      x_ft = column(ind_x,j)
      Z =  np.dot(ind_x,wt)
      h = sigmoid(Z)
      wt[j] = wt[j]-alpha*sum((ind_y-h)*x_ft)-(alpha*c*0.5)*np.sign(wt[j])
  return wt

In [36]:
def l1m_weight_fn(x,y1,y2,y3,y4):
  #wt1 = lor_bgd(x,y1)
  #wt2 = lor_bgd(x,y2)
  #wt3 = lor_bgd(x,y3)

  #wt1 = lor_l2_bgd(x,y1,C)
  #wt2 = lor_l2_bgd(x,y2,C)
  #wt3 = lor_l2_bgd(x,y3,C)

  wt1 = lor_l1_mbgd(x,y1)
  wt2 = lor_l1_mbgd(x,y2)
  wt3 = lor_l1_mbgd(x,y3)
  wt4 = lor_l1_mbgd(x,y4)

  return wt1,wt2,wt3,wt4

In [37]:
ovr_acc = 0
acc1 = 0
acc2 = 0
acc3 = 0
acc4 = 0
for f in range(10):
  X_train = []
  Y_train = []
  #a_outputs = []
  pred_op = []
  p_outputs = []
  arr1 = np.array(subsets[f])
  X_test = arr1.tolist()
  Y_test = y_subsets[f]
  for k in range(10):
    if k!=f:
      arr = np.array(subsets[k])
      list1 = arr.tolist()
      X_train.extend(list1)
      Y_train.extend(y_subsets[k])
  y_tr1,y_tr2,y_tr3,y_tr4 = one_vs_all(Y_train)

  w1,w2,w3,w4 = l1m_weight_fn(X_train,y_tr1,y_tr2,y_tr3,y_tr4)
  for b in range(len(X_test)):
    pred_op = []

    Z1 = np.dot(X_test[b],w1)
    pred_op.append(sigmoid(Z1))
    Z2 = np.dot(X_test[b],w2)
    pred_op.append(sigmoid(Z2))
    Z3 = np.dot(X_test[b],w3)
    pred_op.append(sigmoid(Z3))
    Z4 = np.dot(X_test[b],w4)
    pred_op.append(sigmoid(Z4))
    
    pred_op = np.array(pred_op)
    p_outputs.append(np.argmax(pred_op)+1)
    pred_op = pred_op.tolist()    
  print( 'Fold ' + str(f+1))
  y_actual = pd.Series(Y_test, name= 'Actual' )
  y_pred = pd.Series(p_outputs, name= 'Predicted' )
  confmat = pd.crosstab(y_actual,y_pred)
  print(confmat)
  confmat = np.asarray(confmat)
  class_acc = np.zeros(4)
  for i in range(4):
    num = confmat[i][i]
    s =0
    for j in range(4):
      s += confmat[i][j]
    class_acc[i]= num/s
  #print(class_acc)
  acc = np.trace(confmat)/np.sum(confmat)
  for c in range(4):
    print( 'Accuracy of class'+ str(c+1)+' : ' + str(class_acc[c]*100))
  print( 'Overall Accuracy : ' + str(acc*100))
  ovr_acc += acc
  acc1 += class_acc[0]
  acc2 += class_acc[1]
  acc3 += class_acc[2]
  acc4 += class_acc[3]
  print(p_outputs)
  print(Y_test)
print( 'Average Accuracy : ' + str(ovr_acc*20))
print( 'Average Accuracy of class 1 : ' + str(acc1*20))
print( 'Average Accuracy of class 2 : ' + str(acc2*20))
print( 'Average Accuracy of class 3 : ' + str(acc3*20))
print( 'Average Accuracy of class 4 : ' + str(acc4*20))
  #print(p_outputs)
  #print(Y_test)


Fold 1
Predicted   1   2   3   4
Actual                   
1.0        34   7  30  14
2.0        33  11  41  11
3.0        27  11  22  15
4.0        24  10  41  10
Accuracy of class1 : 40.0
Accuracy of class2 : 11.458333333333332
Accuracy of class3 : 29.333333333333332
Accuracy of class4 : 11.76470588235294
Overall Accuracy : 22.58064516129032
[4, 1, 1, 1, 3, 4, 1, 3, 3, 1, 3, 2, 3, 2, 1, 4, 2, 1, 3, 3, 2, 3, 1, 3, 3, 3, 4, 3, 2, 1, 1, 3, 1, 3, 3, 4, 2, 1, 3, 3, 4, 3, 3, 4, 3, 3, 3, 2, 3, 1, 4, 3, 1, 1, 1, 1, 2, 4, 1, 1, 3, 1, 3, 4, 4, 1, 3, 1, 4, 3, 3, 1, 1, 3, 3, 3, 2, 1, 1, 3, 1, 4, 1, 4, 3, 3, 2, 1, 3, 3, 4, 4, 4, 3, 3, 1, 1, 1, 1, 3, 3, 1, 4, 4, 4, 1, 2, 3, 3, 2, 3, 1, 3, 1, 3, 3, 1, 3, 1, 3, 1, 4, 4, 1, 3, 2, 1, 1, 1, 3, 1, 1, 1, 4, 1, 3, 4, 1, 1, 3, 1, 3, 1, 1, 3, 3, 2, 3, 1, 3, 1, 1, 2, 4, 2, 3, 1, 4, 1, 1, 1, 4, 1, 3, 3, 3, 1, 1, 3, 4, 1, 1, 1, 3, 4, 3, 3, 3, 3, 1, 1, 4, 2, 1, 3, 4, 1, 3, 3, 3, 1, 2, 1, 3, 1, 3, 1, 4, 4, 3, 2, 4, 1, 2, 1, 2, 2, 3, 3, 3, 2, 1, 4, 3, 4, 3, 1, 3, 

In [38]:
def lor_mbgd(x,y):
  alpha = 0.0001
  #c = 0.8286427728546842
  iters =100
  x = np.array(x)
  n = x.shape[1]
  wt = np.ones(n)
  #J=np.zeros((iters,n-1))
  batch_size =20
  #Z = np.dot(x,wt)
  #h = sigmoid(Z)
  for i in range (0,iters):
    randindex = np.random.randint(len(y))
    ind_x = x[randindex:randindex+batch_size]
    ind_y = y[randindex:randindex+batch_size]
    for j in range (0,n-1):
      x_ft = column(ind_x,j)
      Z =  np.dot(ind_x,wt)
      h = sigmoid(Z)
      wt[j] = wt[j]-alpha*sum((ind_y-h)*x_ft)
  return wt

In [39]:
def m_weight_fn(x,y1,y2,y3,y4):
  wt1 = lor_mbgd(x,y1)
  wt2 = lor_mbgd(x,y2)
  wt3 = lor_mbgd(x,y3)
  wt4 = lor_mbgd(x,y4)

  #wt1 = lor_l2_bgd(x,y1,C)
  #wt2 = lor_l2_bgd(x,y2,C)
  #wt3 = lor_l2_bgd(x,y3,C)

  #wt1 = lor_l1_bgd(x,y1,C)
  #wt2 = lor_l1_bgd(x,y2,C)
  #wt3 = lor_l1_bgd(x,y3,C)
  return wt1,wt2,wt3,wt4

In [40]:
ovr_acc = 0
acc1 = 0
acc2 = 0
acc3 = 0
acc4 = 0
for f in range(10):
  X_train = []
  Y_train = []
  #a_outputs = []
  pred_op = []
  p_outputs = []
  arr1 = np.array(subsets[f])
  X_test = arr1.tolist()
  Y_test = y_subsets[f]
  for k in range(10):
    if k!=f:
      arr = np.array(subsets[k])
      list1 = arr.tolist()
      X_train.extend(list1)
      Y_train.extend(y_subsets[k])
  y_tr1,y_tr2,y_tr3,y_tr4 = one_vs_all(Y_train)

  w1,w2,w3,w4 = m_weight_fn(X_train,y_tr1,y_tr2,y_tr3,y_tr4)
  for b in range(len(X_test)):
    pred_op = []

    Z1 = np.dot(X_test[b],w1)
    pred_op.append(sigmoid(Z1))
    Z2 = np.dot(X_test[b],w2)
    pred_op.append(sigmoid(Z2))
    Z3 = np.dot(X_test[b],w3)
    pred_op.append(sigmoid(Z3))
    Z4 = np.dot(X_test[b],w4)
    pred_op.append(sigmoid(Z4))
    
    pred_op = np.array(pred_op)
    p_outputs.append(np.argmax(pred_op)+1)
    pred_op = pred_op.tolist()    
  print( 'Fold ' + str(f+1))
  y_actual = pd.Series(Y_test, name= 'Actual' )
  y_pred = pd.Series(p_outputs, name= 'Predicted' )
  confmat = pd.crosstab(y_actual,y_pred)
  print(confmat)
  confmat = np.asarray(confmat)
  class_acc = np.zeros(4)
  for i in range(4):
    num = confmat[i][i]
    s =0
    for j in range(4):
      s += confmat[i][j]
    class_acc[i]= num/s
  #print(class_acc)
  acc = np.trace(confmat)/np.sum(confmat)
  for c in range(4):
    print( 'Accuracy of class'+ str(c+1)+' : ' + str(class_acc[c]*100))
  print( 'Overall Accuracy : ' + str(acc*100))
  ovr_acc += acc
  acc1 += class_acc[0]
  acc2 += class_acc[1]
  acc3 += class_acc[2]
  acc4 += class_acc[3]
  print(p_outputs)
  print(Y_test)
print( 'Average Accuracy : ' + str(ovr_acc*20))
print( 'Average Accuracy of class 1 : ' + str(acc1*20))
print( 'Average Accuracy of class 2 : ' + str(acc2*20))
print( 'Average Accuracy of class 3 : ' + str(acc3*20))
print( 'Average Accuracy of class 4 : ' + str(acc4*20))
  #print(p_outputs)
  #print(Y_test)


Fold 1
Predicted   1   2   3   4
Actual                   
1.0        19  12  34  20
2.0        23  15  36  22
3.0        18  11  23  23
4.0        17  18  29  21
Accuracy of class1 : 22.35294117647059
Accuracy of class2 : 15.625
Accuracy of class3 : 30.666666666666664
Accuracy of class4 : 24.705882352941178
Overall Accuracy : 22.87390029325513
[4, 1, 1, 4, 3, 4, 2, 3, 3, 2, 3, 2, 3, 3, 1, 4, 1, 1, 4, 2, 4, 3, 1, 2, 4, 3, 1, 3, 2, 2, 1, 3, 1, 3, 3, 4, 2, 1, 2, 4, 2, 4, 4, 4, 4, 3, 2, 3, 3, 3, 3, 2, 2, 4, 1, 1, 2, 4, 4, 1, 2, 1, 3, 4, 4, 1, 3, 1, 4, 3, 4, 1, 1, 4, 3, 3, 4, 3, 4, 3, 1, 4, 1, 1, 4, 3, 3, 2, 3, 3, 1, 1, 4, 3, 3, 4, 2, 1, 2, 4, 3, 1, 4, 1, 2, 3, 3, 4, 3, 3, 1, 1, 4, 1, 3, 3, 1, 3, 1, 3, 1, 4, 4, 2, 3, 3, 2, 3, 1, 2, 1, 1, 2, 1, 2, 4, 1, 2, 2, 1, 1, 4, 3, 2, 3, 3, 4, 3, 4, 3, 1, 3, 3, 4, 3, 3, 2, 4, 2, 1, 3, 1, 2, 3, 3, 3, 1, 2, 4, 1, 3, 1, 3, 4, 4, 4, 3, 3, 3, 1, 1, 4, 4, 2, 3, 4, 1, 4, 3, 3, 1, 3, 3, 3, 3, 3, 1, 4, 4, 4, 3, 4, 1, 2, 1, 3, 3, 3, 4, 3, 4, 3, 4, 3, 1, 3, 2, 3